# 問題
文章textに係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

In [2]:
import spacy


def analyze_dependencies(text):
    # 日本語のモデルをロード
    nlp = spacy.load("ja_ginza")

    # テキストを解析
    doc = nlp(text)
    print("doc;",doc)
    # 係り受け関係を抽出
    for token in doc:
        if token.dep_ != "ROOT":  # ROOTは係り先がないので除外
            # 係り元と係り先のテキストを取得
            source = token.text
            target = token.head.text
            # タブ区切りで出力
            print(f"{source}\t{target}")


# テスト用のテキスト
text = """メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。"""

# 係り受け解析を実行
analyze_dependencies(text)

doc; メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
メロス	激怒
は	メロス
し	激怒
た	激怒
。	激怒
必ず	除か
、	必ず
かの	暴虐
邪智	暴虐
暴虐	王
の	暴虐
王	除か
を	王
除か	決意
なけれ	除か
ば	なけれ
なら	なけれ
ぬ	なけれ
と	除か
し	決意
た	決意
。	決意

	メロス
メロス	わから
に	メロス
は	メロス
政治	わから
が	政治
ぬ	わから
。	わから

	メロス
メロス	牧人
は	メロス
、	メロス
村	牧人
の	村
で	牧人
ある	で
。	牧人

	笛
笛	吹き
を	笛
吹き	暮し
、	吹き
羊	遊ん
と	羊
遊ん	暮し
で	遊ん
て	暮し
来	て
た	暮し
。	暮し

	邪悪
けれど	

も	

邪悪	敏感
に	邪悪
対し	に
ては	に
、	邪悪
人	倍
一	倍
倍	敏感
に	倍
で	敏感
あっ	で
た	敏感
。	敏感


In [9]:
import spacy
import ginza

nlp = spacy.load("ja_ginza")

def analyze_dependencies_bunsetu(text: str):
    doc = nlp(text)

    # 文節スパンを取得
    bunsetsu_spans = list(ginza.bunsetu_spans(doc))

    # トークン index → 文節 index のマッピング
    token2span = {}
    for i, sp in enumerate(bunsetsu_spans):
        for t in sp:
            token2span[t.i] = i

    # 文節→文節の依存を出力
    for i, sp in enumerate(bunsetsu_spans):
        head_tok = sp.root.head
        head_idx = token2span.get(head_tok.i)
        if head_idx is None or head_idx == i:
            continue  # 係り先がない or 自分自身はスキップ
        src = sp.text.replace(" ", "")
        dst = bunsetsu_spans[head_idx].text.replace(" ", "")
        print(f"{src}\t{dst}")


# テスト
text = """メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。"""

analyze_dependencies_bunsetu(text)

メロスは	激怒した。
必ず、	除かなければならぬと
かの	邪智暴虐の
邪智暴虐の	王を
王を	除かなければならぬと
除かなければならぬと	決意した。

メロスには	わからぬ。
政治が	わからぬ。

メロスは、	牧人である。
村の	牧人である。

笛を	吹き、
吹き、	暮して来た。
羊と	遊んで
遊んで	暮して来た。

けれども	邪悪に対しては、
邪悪に対しては、	敏感であった。
人一倍に	敏感であった。
